In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
import matplotlib.pyplot as plt
from keras import optimizers
tf.__version__

Using TensorFlow backend.


'2.2.0-rc2'

In [3]:
  
#1. Read the dataset
df = pd.read_csv("bank.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()
# There are 3 features which ate of object data type which we will be converting to numeric in further steps.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()
# There are no missing values.

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df['Exited'].value_counts()

# Observing a class imbalance. We have more data points for customers who have not exited than who have exited.

0    7963
1    2037
Name: Exited, dtype: int64

In [7]:
#2. Drop the columns which are unique for all users like IDs (2.5 points)

#Dropping columns ID and Rownumber, Surname

df = df.drop(columns = ['RowNumber','CustomerId','Surname'], axis = 1)
df.shape

(10000, 11)

In [0]:
# Converting Categorical variable to numeric
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
le= LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Geography'] = le.fit_transform(df['Geography'])

In [9]:
# Geography and Gender columns both have converted to numeric. And RowNumber, CustomerId, Surname are dropped from teh dataframe
df.sample(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6619,647,0,1,32,5,97041.16,1,1,1,23132.73,0
7860,659,0,1,32,9,95377.13,1,0,1,187551.24,0
2900,626,2,0,26,8,148610.41,3,0,1,104502.02,1
7834,703,0,1,56,9,0.00,1,0,0,85547.33,1
9044,560,2,1,47,1,0.00,1,0,0,128882.66,1
8294,704,2,0,36,2,175509.80,2,1,0,152039.67,0
1595,677,0,1,38,4,0.00,2,1,0,187800.63,0
6510,619,2,1,46,8,62400.48,1,1,1,132498.39,1
4698,732,0,1,62,2,0.00,2,1,1,25438.87,0
1260,704,1,1,27,5,147004.34,1,1,0,64381.33,1


In [0]:
#3. Distinguish the feature and target set
X = df.drop(columns = 'Exited', axis = 1)
Y = df['Exited']

In [15]:
X.sample(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
1697,683,0,1,30,8,110829.52,2,0,0,24938.84
9729,651,0,1,23,10,0.00,2,1,1,170099.23


In [17]:
Y.sample(2)

9692    0
9885    0
Name: Exited, dtype: int64

In [19]:
#4. Divide the data set into Train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=43, stratify = Y)
X_train.shape

(8000, 10)

In [0]:
#5. Normalize the train and test data (2.5 points)
# normalising using standard scalar
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)
Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

In [0]:
# Converting Y_train and Y_test to array

In [24]:
#6. Initialize & build the model (7.5 points)
#Initialize Sequential Graph (model)

from keras.models import Sequential
from keras.layers import Dense

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(activation = 'relu', input_dim = 10, units=6))
model.add(tf.keras.layers.Dense(20, activation = 'relu')) 
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid')) 
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train_std, Y_train, validation_data=(X_test_std,Y_test),epochs=100,batch_size=32)

# This model is a good fit as it has both train & test accuracy averaging around 83% to 85%.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 66        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5396 - accuracy: 0.7806 - val_loss: 0.5109 - val_accuracy: 0.7965
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4966 - accuracy: 0.7962 - val_loss: 0.4895 - val_accuracy: 0.7965
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4807 - accuracy: 0.7962 -

In [25]:
#7. Optimize the model (5 points)
from keras import optimizers
sgd = tf.keras.optimizers.SGD(lr=0.001)  # With learning rate 
#Compile the model
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train_std, Y_train, validation_data=(X_test_std,Y_test),epochs=100,batch_size=32)

# After optimising with a reduced learning rate, train accuracy remains arouund 85% & test accuracy is around 87%

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 66        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3520 - accuracy: 0.8540 - val_loss: 0.3419 - val_accuracy: 0.8705
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3517 - accuracy: 0.8544 - val_loss: 0.3418 - val_accuracy: 0.8700
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3515 - accuracy: 0.8544 -

In [26]:
#9. Predict the results using 0.5 as a threshold (5 points)
y_pred = model.predict(X_test_std)
y_pred = (y_pred > 0.5)
y_pred.shape

(2000, 1)

In [27]:
from sklearn import metrics
test_loss, test_acc = model.evaluate(x=X_test_std,y=Y_test)
train_loss, train_acc = model.evaluate(x=X_train_std,y=Y_train)
print("training accuracy:", train_acc)
print("training loss:", train_loss)
print("test accuracy:", test_acc)
print("test loss:", test_loss)

conf_mat = metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print(metrics.classification_report(Y_test, y_pred))

250/250 [==============================] - 0s 1ms/step - loss: 0.3491 - accuracy: 0.8547
training accuracy: 0.8547499775886536
training loss: 0.34905022382736206
test accuracy: 0.8730000257492065
test loss: 0.33976951241493225
Confusion matrix:
 [[1538   55]
 [ 199  208]]
              precision    recall  f1-score   support

           0       0.89      0.97      0.92      1593
           1       0.79      0.51      0.62       407

    accuracy                           0.87      2000
   macro avg       0.84      0.74      0.77      2000
weighted avg       0.87      0.87      0.86      2000



In [0]:
#Even though we are seeing a good train & test accuracy,by looking at confusion matrix, 
# we have to reduce the number of False positives i.e the model predicted them as customers which will not exit but they exited. 
# # it was expected to provide better results for class 0 / continuing cutomers class.